## RAG Retrieval

## k_count
Количество возвращаемых retrieval фрагментов.

Выбор параметров для поиска и настройки `Retriever` зависит от конкретной задачи и требований к системе. Вот несколько рекомендаций по выбору `search_type`, значений параметров `k`, `lambda_mult` и других параметров:

1. **`search_type`**:
    - **`similarity`**: Используйте этот тип поиска, если вам нужно найти документы, которые наиболее похожи на запрос. Это стандартный выбор для большинства задач.
    - **`mmr`**: Используйте максимальную маргинальную релевантность (MMR) для увеличения разнообразия возвращаемых результатов. Этот тип поиска полезен, когда нужно избегать получения слишком похожих документов.
    - **`similarity_score_threshold`**: Используйте этот тип поиска, если вам важно, чтобы возвращаемые документы превышали определенный порог релевантности. Это может быть полезно для задач, требующих высоко релевантных ответов.

2. **`k` (количество возвращаемых документов)**:
    - Значение по умолчанию составляет 4, но оптимальное значение может варьироваться в зависимости от задачи. Если требуется больше контекста или более полное покрытие темы, можно увеличить `k`. Например, `k=5` или `k=10`.
    - Для более точных запросов или ограниченного объема данных может быть достаточно меньшего значения, например, `k=3`.

3. **`lambda_mult` (только для `mmr` поиска)**:
    - Этот параметр регулирует разнообразие результатов. Значение по умолчанию составляет 0.5, что является компромиссом между релевантностью и разнообразием.
    - Для максимального разнообразия установите `lambda_mult` ближе к 0. Для минимального разнообразия, но максимальной релевантности установите `lambda_mult` ближе к 1.
    - Оптимальное значение зависит от задачи: для широкого охвата информации можно установить `lambda_mult=0.25`, а для узконаправленных запросов – ближе к 0.75.

4. **Другие параметры**:
    - **`fetch_k`**: Этот параметр определяет количество документов, которые передаются алгоритму MMR для выбора наиболее релевантных и разнообразных результатов. Значение по умолчанию — 20, и его можно оставить без изменений, если нет специфических требований.
    - **`score_threshold`**: Устанавливайте только для `similarity_score_threshold` типа поиска. Зависит от порога релевантности, который вы хотите задать.
    - **`filter`**: Используйте для фильтрации документов по метаданным, если у вас есть дополнительные критерии отбора.



#### Значение `score_threshold`:

- **Низкое значение (< 0.3)**: Используйте, если вы хотите получить как можно больше результатов, даже если они могут быть менее релевантными. Подходит для задач, где важен широкий охват информации.
- **Среднее значение (0.3 - 0.7)**: Используйте для достижения баланса между количеством и качеством результатов. Подходит для задач, где требуется умеренная точность и достаточное количество результатов.
- **Высокое значение (> 0.7)**: Используйте, если вам нужны только высоко релевантные результаты. Подходит для задач, где важна высокая точность, и количество результатов может быть ограничено.


![structure.png](../Schemes/Retrieval_serch_type.png)

### Разбиваем документы

In [ ]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Retrieval_testing_k_3_html_split"

In [ ]:
# получаем смысловую разбивку всех документов
# splits = split_documents_my_html()
splits = split_documents_html(chunk_size=1000, chunk_overlap=100)
# splits = split_documents_standart(chunk_size=1000, chunk_overlap=100)

# индексируем
embeddings_model = GigaChatEmbeddings(scope="GIGACHAT_API_PERS", verify_ssl_certs=False)
db = Chroma.from_documents(documents=splits, embedding=embeddings_model)

# подключаем llm
llm = GigaChat(model='GigaChat-Plus', verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

### Устанавливаем ретриевер для поиска контекста

In [ ]:
# ________________________ 1 _________________________
retriever_similarity_1 = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 1}
)

retriever_mmr_1 = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1, 'lambda_mult': 0.25}
)

retriever_similarity_score_threshold_1 = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 1, 'score_threshold': 0.7}
)

# ________________________ 3 _________________________

retriever_similarity_3 = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

retriever_mmr_3 = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 3, 'lambda_mult': 0.25}
)

retriever_similarity_score_threshold_3 = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 3, 'score_threshold': 0.7}
)

# ________________________ 7 _________________________

retriever_similarity_7 = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 7}
)

retriever_mmr_7 = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 7, 'lambda_mult': 0.25}
)

retriever_similarity_score_threshold_7 = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.6},
    k=7
)

In [ ]:
prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении поиска ответов на вопросы. Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. Если вы не знаете ответа, просто скажите, что вы не знаете.
        Test_name: retriever_mmr_3
        Question: {question} 
        Context: {context} 
        Answer:""")


rag_chain = (
    {"context": retriever_similarity_3, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
answer_1 = rag_chain.invoke("При принятии пользовательских соглашений, кому еще попадают мои данные?")
answer_2 = rag_chain.invoke("Я не пользовалась кредиткой год, но у меня там лежат мои 800 рублей, я могу их как-то вернуть?")
answer_3 = rag_chain.invoke("Какие документы потребуются, если открывать счет для нотариуса, занимающегося частной практикой? Может ли нотариус быть иностранцем при этом?")
answer_4 = rag_chain.invoke("Какой минимум страховой выплаты мне вернут, если моя страховая сумма = 50 000 долларов?")

### Выводы
Большое количестов контекста не влияет положительно на результат, оптимальное колличесвто - 3.  

Метод similarity_score_threshold не показал себя - опустим его

## k_count_3_similarity

In [ ]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "Retrieval_testing_k_count_3_similarity_html_split"

In [ ]:
retriever_similarity_3 = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении поиска ответов на вопросы. Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. Если вы не знаете ответа, просто скажите, что вы не знаете.
        Test_name: retriever_similarity_3
        Question: {question} 
        Context: {context} 
        Answer:""")


rag_chain = (
    {"context": retriever_similarity_3, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Загрузка Датасета

In [ ]:
import pandas as pd
df = pd.read_csv('../dataset.csv')

df['answer'] = df['question'].apply(lambda x: rag_chain.invoke(x.strip()))
display(df)

In [ ]:
df.to_csv('../rag_answer/rag_step_retriever/html_split/rag_3_similarity.csv', index=False)

## Re-Rank from Cohere

Сравнение ContextualCompressionRetriever и co.rerank

Функция `compression_retriever.get_relevant_documents(query)` и функция `co.rerank()` имеют разные цели и используют разные подходы для обработки и ранжирования документов.

## Variant 1 compressor = CohereRerank()

### ContextualCompressionRetriever

`ContextualCompressionRetriever` используется для получения релевантных документов с применением компрессии. Компрессия здесь означает отбор наиболее значимых частей из документов для ответа на конкретный запрос. `CohereRerank` используется как компрессор для выбора и компрессии этих частей.

### Пример работы ContextualCompressionRetriever:

1. **Поиск документов**: Исходный `retriever` используется для поиска документов по запросу.
2. **Компрессия документов**: Компрессор (в данном случае `CohereRerank`) применяется для извлечения наиболее релевантных частей из найденных документов.
3. **Возвращение документов**: Компрессор возвращает сжатые версии документов, которые считаются наиболее релевантными для запроса.

In [2]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "TEST_FINAL_COMPRESSOR"

In [4]:
# OpenAI 
llm=ChatOpenAI(model_name="gpt-3.5-turbo-0125")
embeddings_model = OpenAIEmbeddings()

In [9]:
# получаем смысловую разбивку всех документов
# splits = split_documents_my_html()
splits = split_documents_html(chunk_size=1000, chunk_overlap=100)
# splits = split_documents_standart(chunk_size=1000, chunk_overlap=100)

# индексируем
# embeddings_model = GigaChatEmbeddings(scope="GIGACHAT_API_PERS", verify_ssl_certs=False)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

# подключаем llm
# llm = GigaChat(model='GigaChat-Plus', verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [20]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 25}
)

# CohereRerank
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

prompt = ChatPromptTemplate.from_template("""Вы являетесь помощником в выполнении поиска ответов на вопросы. Используйте приведенные ниже фрагменты извлеченного контекста, чтобы ответить на вопрос. Если вы не знаете ответа, просто скажите, что вы не знаете.
        Question: {question} 
        Context: {context} 
        Answer:""")


rag_chain_compressor = (
    {"context": compression_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain_compressor.invoke("Какие документы потребуются, если открывать счет для нотариуса, занимающегося частной практикой? Может ли нотариус быть иностранцем при этом?")
# response = rag_chain_compressor.invoke("На какой срок выдается эта доверенность? Можно ли ее передоверить другому лицу?")
response
# rag_chain.invoke("При принятии пользовательских соглашений, кому еще попадают мои данные?")

'Для открытия счета для нотариуса, занимающегося частной практикой, потребуются следующие документы:\n1. Документ, удостоверяющий личность.\n2. Документ, подтверждающий наделение нотариуса полномочиями (назначение на должность), выдаваемый органами юстиции субъектов Российской Федерации.\n3. Адрес места жительства (регистрации) или места пребывания.\n4. Контактная информация.\n5. Выписка из ЕГРИП.\n6. Информация о страховом номере индивидуального лицевого счета застрахованного лица в системе.\n\nНотариус может быть иностранцем, при этом ему также потребуется предоставить документ, подтверждающий его право на пребывание (проживание) в Российской Федерации, если это предусмотрено законодательством.'

### Загрузка Датасета

In [ ]:
import pandas as pd
import time

# Загрузка данных
df = pd.read_csv('../dataset.csv')

# Функция для обработки одного запроса с учетом задержки
def process_question(question):
    answer = rag_chain_compressor.invoke(question.strip())
    time.sleep(3)  # Ждать n секунд перед следующим запросом
    return answer

# Применение функции к каждому вопросу
df['answer'] = df['question'].apply(process_question)

# Отображение результатов
display(df)


### Время - 6 минут 43 секунды

In [ ]:
df.to_csv('../rag_answer/rag_step_retriever/html_split/rag_compressor_rerank.csv', index=False)

## Variant 2 co.rerank
![structure.png](../Schemes/Retrieval_rerank.png)


`co.rerank` используется для повторного ранжирования списка документов на основе их релевантности к запросу. Этот метод обычно применяется после первоначального поиска, чтобы улучшить порядок документов по их значимости.

### Пример работы co.rerank:

1. **Поиск документов**: Сначала документы ищутся с помощью `retriever`.
2. **Повторное ранжирование**: `co.rerank` применяется к найденным документам для их повторного упорядочивания на основе релевантности к запросу.
3. **Возвращение документов**: Возвращаются документы в новом порядке, основанном на результатах повторного ранжирования.

### Библиотеки

In [1]:
from necessity import *

# LangSmith
os.environ["LANGCHAIN_PROJECT"] = "TEST_FINAL_RERANK"

In [3]:
# получаем смысловую разбивку всех документов
# splits = split_documents_my_html()
splits = split_documents_html(chunk_size=1000, chunk_overlap=100)
# splits = split_documents_standart(chunk_size=1000, chunk_overlap=100)

# индексируем
# embeddings_model = GigaChatEmbeddings(scope="GIGACHAT_API_PERS", verify_ssl_certs=False)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

In [29]:
# # подключаем llm
# llm = GigaChat(model='GigaChat-Plus', verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [30]:
# получаем только уникальные документы из всего списка
def get_unique_documents(documents):
    unique_documents = []
    seen_contents = set()

    for doc in documents:
        content = doc.page_content
        if content not in seen_contents:
            unique_documents.append(doc)
            seen_contents.add(content)

    return unique_documents

# по списку документов получаем определенный с page_content=target_page_content
def get_document_by_content(documents, target_page_content):
    for doc in documents:
        if doc.page_content == target_page_content:
            return doc
        

# Определяем функцию rerank_doc
def rerank_doc(query: str, top_n: int):
    retrieved_docs = retriever.invoke(query)
    unique_docs = get_unique_documents(retrieved_docs)
    
    # Rerank documents
    rerank_results = co.rerank(
        query=query, 
        documents=[doc.page_content for doc in unique_docs], 
        top_n=top_n, 
        model="rerank-multilingual-v3.0",
        return_documents=True
    )
    
    reranked_docs = []
    
    for info in rerank_results.results:
        doc = get_document_by_content(unique_docs, info.document.text)
        doc.metadata['relevance_score'] = info.relevance_score
        reranked_docs.append(doc)
        
    return reranked_docs

# Определяем функцию для ранжирования документов
def rerank_retriever(query):
    return rerank_doc(query, top_n=4)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 25}
)

# Определяем шаблон для запроса
prompt = ChatPromptTemplate.from_template("""
        Вы являетесь экспертом в области знаний пользовательских соглашений для банка Tinkoff и помогаете отвечать на вопросы.\n
        Я собираюсь задать вам вопрос. Ваш ответ должен быть исчерпывающим и основываться на следующем контексте (контекст хранится в Context), если он уместен. \n
        Отвечайте от лица представителя Банка и не уходите от темы, ответ должен быть не коротким, а полным.
        
        Question: {question} 
        Context: {context} 
        Answer:""")


# Создаем новую RAG цепочку с использованием rerank_retriever
rag_chain_rerank = (
    {"context": rerank_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Пример использования новой цепочки
# response = rag_chain_rerank.invoke("Какие документы потребуются, если открывать счет для нотариуса, занимающегося частной практикой? Может ли нотариус быть иностранцем при этом?")
# response = rag_chain_rerank.invoke("Какой минимум страховой выплаты мне вернут, если моя страховая сумма = 50 000 долларов?")
# print(response)

In [34]:
response = rag_chain_rerank.invoke("Какие документы потребуются, если открывать счет для нотариуса, занимающегося частной практикой? Может ли нотариус быть иностранцем при этом?")
# response = rag_chain_rerank.invoke("Какой минимум страховой выплаты мне вернут, если моя страховая сумма = 50 000 долларов?")
print(response)

Для открытия счета нотариуса, занимающегося частной практикой, необходимо предоставить следующие документы:

1. Документ, удостоверяющий личность и указанный в пункте 1.4 настоящего Перечня.
2. Документ, подтверждающий наделение нотариуса полномочиями (назначение на должность), выдаваемый органами юстиции субъектов Российской Федерации, в соответствии с законодательством Российской Федерации.
3. Адрес места жительства (регистрации) или места пребывания (если его нельзя установить из документа, удостоверяющего личность, а также если адрес места жительства (регистрации) или места пребывания не совпадают).
4. Контактная информация (например, номер телефона, факса, адрес электронной почты, почтовый адрес (при наличии).
5. Выписка из ЕГРИП. Банком может быть принята копия выписки, заверенная регистрирующим органом. Выписка считается действительной для предоставления в Банк в течение 30 (тридцати) календарных дней с даты ее выдачи и должна содержать актуальные сведения на дату ее предоставле

### Вывод как меняются индексы 

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 25}
)

def compare(query: str, top_n: int):
    retrieved_docs = retriever.invoke(query)
    
    print(f"Unique documents count: {len(retrieved_docs)}")
    
    print('Query:', query)
    # добавляем индексацию в метадату
    for idx, doc in enumerate(retrieved_docs):
        doc.metadata["initial_position"] = idx

    # Create a mapping from document content to its position
    content_to_initial_position = {doc.page_content: doc.metadata["initial_position"] for doc in retrieved_docs}
    
    # Rerank documents
    rerank_results = co.rerank(
        query=query, 
        documents=[doc.page_content for doc in retrieved_docs], 
        top_n=top_n, 
        model="rerank-multilingual-v3.0",
        return_documents=True
    )
    
    # Compare order change
    original_docs = []
    reranked_docs = []
    
    for new_position, info in enumerate(rerank_results.results):
        original_position = content_to_initial_position[info.document.text]
        print(f"{original_position} -> {new_position}")
        if new_position != original_position:
            original_docs.append([original_position, info.document.text])
            reranked_docs.append([new_position, retrieved_docs[new_position].page_content])
        

    for orig, rerank in zip(original_docs, reranked_docs):
        print(f"ORIGINAL {orig[0]}\n{orig[1]}\nRERANKED {rerank[0]}\n{rerank[1]}\n---------------")
        
    return rerank_results

a = compare("Какой минимум страховой выплаты мне вернут, если моя страховая сумма = 50 000 долларов?", 25)
a

### Загрузка Датасета

Giga generation stopped with reason: blacklist
ReadTimeout: The read operation timed out

In [ ]:
import pandas as pd
import time

# Загрузка данных
df = pd.read_csv('../dataset.csv')

# # Функция для обработки одного запроса с учетом задержки
# def process_question(question):
#     answer = rag_chain_rerank.invoke(question.strip())
#     # time.sleep(3)  # Ждать n секунд перед следующим запросом
#     return answer

# # Применение функции к каждому вопросу
# df['answer'] = df['question'].apply(process_question)

df['answer'] = df['question'].apply(lambda x: rag_chain_rerank.invoke(x.strip()))

# Отображение результатов
display(df)

In [ ]:
display(df)

In [14]:
df.to_csv('../rag_answer/rag_step_retriever/html_split/rag_rerank_gpt_3.5.csv', index=False)

### Сравнение двух подходов

- **`compression_retriever.get_relevant_documents(query)`**: Этот метод не просто повторно ранжирует документы, а также извлекает и сжимает наиболее важные части каждого документа, предоставляя сжатую, но релевантную информацию.
- **`co.rerank(query, documents)`**: Этот метод сохраняет оригинальные документы и только изменяет порядок их следования на основе их релевантности к запросу.